
## CNN text classification model

auther: kun wu

### Config keras for GPU

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [ ]:
# import tensorflow as tf
# from keras import backend as K

# with K.tf.device('/gpu:1'):
#     config = tf.ConfigProto(intra_op_parallelism_threads=4,\
#            inter_op_parallelism_threads=4, allow_soft_placement=True,\
#            device_count = {'CPU' : 1, 'GPU' : 1})
#     session = tf.Session(config=config)
#     K.set_session(session)

### Load data

In [2]:
import os
import pandas as pd

BASE_DIR = '/home/kwu14/data/cs584_course_project/'

train_df = pd.read_csv(os.path.join(BASE_DIR, 'preprocessed_train.csv'))

train_df.head()

,Unnamed: 0,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,0,59848,0,cool like would want mother read realli great ...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
1,1,59849,0,thank would make life lot less anxietyinduc ke...,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
2,2,59852,0,urgent design problem kudo take impress,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
3,3,59855,0,someth ill abl instal site releas,0.000000,0.0,0.000000,0.00000,0.0,NaN,...,2006,rejected,0,0,0,0,0,0.0,0,4
4,4,59856,1,haha guy bunch loser,0.021277,0.0,0.021277,0.87234,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.0,4,47


In [3]:
x_train = train_df['comment_text'].astype(str).values
y_train = train_df['target'].values

In [4]:
%%time
from keras.preprocessing import text

tk = text.Tokenizer()
tk.fit_on_texts(x_train)
x_train = tk.texts_to_sequences(x_train)

vocab_size = len(tk.word_index)
max_len = 0
for tokens in x_train:
    max_len = len(tokens) if len(tokens) > max_len else max_len

print(f'vocabulary size: {vocab_size}, max length in the text is {max_len}')

Using TensorFlow backend.
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/kwu14/anaconda

vocabulary size: 392342, max length in the text is 305
CPU times: user 2min 14s, sys: 984 ms, total: 2min 15s
Wall time: 2min 21s


In [5]:
from keras.preprocessing import sequence

x_train = sequence.pad_sequences(x_train, maxlen=max_len)

In [6]:
from glovevectorizer import generate_weights, load_glove_weights

path = os.path.join(BASE_DIR, 'glove.6B.300d.txt')
glove_weights = load_glove_weights(path)
weights = generate_weights(glove_weights, word2idx=tk.word_index)

In [20]:
from keras.layers import Input, Conv1D, MaxPooling1D, Embedding, Flatten, Dense
from keras.models import Model


embedding_size = 300

words = Input(shape=(max_len, ), name='input_layer')
x = Embedding(vocab_size + 1, embedding_size, weights=[weights], trainable=False, name='embedding_layer')(words)
l_cov1= Conv1D(128, 5, activation='relu')(x)
l_pool1 = MaxPooling1D(5)(l_cov1)
l_cov2 = Conv1D(128, 5, activation='relu')(l_pool1)
l_pool2 = MaxPooling1D(5)(l_cov2)
l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
l_pool3 = MaxPooling1D(5)(l_cov3)  # global max pooling
l_flat = Flatten()(l_pool3)
l_dense = Dense(128, activation='relu')(l_flat)
out = Dense(1, activation='sigmoid')(l_dense)

model = Model(input=words, output=out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_layer (InputLayer)     (None, 305)               0         
_________________________________________________________________
embedding_layer (Embedding)  (None, 305, 300)          117702900 
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 301, 128)          192128    
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 60, 128)           0         
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 56, 128)           82048     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 11, 128)           0         
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 7, 128)            82048     
__________

/home/kwu14/anaconda3/envs/p37-nlp/lib/python3.7/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [ ]:
epochs = 10
batch_size = 1024

history = model.fit(
    x_train, y_train, 
    batch_size=batch_size, 
    epochs=epochs, 
    validation_split=0.2
)

Train on 1443899 samples, validate on 360975 samples
Epoch 1/10
1443899/1443899 [==============================] - 71s 49us/step - loss: 0.2150 - acc: 0.9425 - val_loss: 0.2343 - val_acc: 0.9370
Epoch 2/10
1443899/1443899 [==============================] - 71s 49us/step - loss: 0.2117 - acc: 0.9433 - val_loss: 0.2391 - val_acc: 0.9368
Epoch 3/10
 307200/1443899 [=====>........................] - ETA: 49s - loss: 0.2073 - acc: 0.9445